In [3]:
# Importações necessárias
from golden import *
import pandas as pd
import numpy as np
from datetime import datetime

# Configurações de exibição do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Iniciando o client para acessar o BQ
client = config_client('prod')

# Verificando se o client está conectado
check_client_connection(client)

# Declarando a query que vamos utilizar
query = """

-- BASE NEUROTECH (HISTÓRICO) => Ajustes abaixos feitos em 02/04/2024
-- AJUSTES: 1. AJUSTE NA FL DE RECURSO (OLHAR PARA O FUNIL) = Feito!
--        : 2. MARCAR O QUE FOI AUDITADO (OLHAR PARA O FUNIL) = Feito!
--        : 3. GARANTIR QUE A FL_GLOSADO TENHA MESMA INFORMAÇÃO QUE O DASH DA NEUROTECH (DS_MOTIVO_GLOSA DEVE BALIZAR ESSA FLAG), ou seja, paramos de olhar para o funil nos acompanhamentos da Neurotech e depois arrumamos no oficial

with
atendimentos_recursados as (
  -- Onde pegamos todos os atendimentos que foram recursados
  -- Pegamos dessa tabela e não da de log, visto que a de log só armazena alterações que o time de auditoria faz e o recurso parte do prestador
  select
    ptr.partner_transfer_id as id_partner_transfer,
    min(ptr.created_at) as dt_recurso
  from petlove-dataeng-prod-01.curated_health.health_partner_transfer_request ptr
  where ptr.refdate <= current_date()
  group by 1
),
cnpj_cpf_empresa_titular as (
  -- Nessa CTE pegaremos o CNPJ da empresa B2B2E
  select distinct
    usp.id as id_pet,
    usp.sponsor_id as id_empresa,
    us.st_name as nome_empresa,
    us.st_cnpj as cnpj
  from petlove-dataeng-prod-01.curated_health.health_user_pet usp
  left join petlove-dataeng-prod-01.curated_health.health_user us
    on usp.sponsor_id = us.id
    and us.refdate <= current_date()
  where usp.refdate <= current_date()
    and usp.sponsor_id is not null
    and trim(us.st_name) <> 'Treinamento Petlove' -- Trazem o CNPJ nulo!
),
base_classificacao_clinicas as (
  -- CTE com classificação das clínicas
  select distinct
    chc.user_id as id_prestador,
    string_agg(distinct cc.display_name, ' | ') as classificacao
  from curated_health.health_clinic_has_category chc
  left join curated_health.health_clinic_category cc
    on chc.clinic_category_id = cc.id
    and cc.refdate <= current_date()
  where chc.refdate <= current_date()
  group by chc.user_id
),
final_0 as (
  -- Base principal onde pegamos a maioria dos dados e juntamos com as CTE's (Tabela base: custos_medicos)
  select
    cm.empresa,
    cm.sistema_origem_custo,
    ben.nome_empresa as empresa_titular,
    ccet.cnpj as CNPJ_empresa_titular,
    ben.dt_inclusao_pet as dt_compra,
    cm.cpf_tutor as cpf_tutor,
    cm.cpf_tutor as cpf_tutor_hash,
    lower(cm.cidade_tutor) as cidade_tutor,
    lower(cm.estado_tutor) as estado_tutor,
    ben.tutor_cep as tutor_cep,
    cm.id_pet_bu,
    ben.dt_nascimento_pet,
    ben.idade_pet_anos,
    ben.especie,
    ben.nome_pet,
    coalesce(ben.raca, 'N/I') as raca_pet,
    coalesce(ben.genero_pet, 'N/I') as genero_pet,
    cm.id_plano,
    cm.nome_plano,
    cm.nome_plano_dts_pet,
    re.state_slug as uf_plano,
    re.name as regiao_plano,
    pp.vl_price_month as valor_mensal_plano,
    cm.portifolio as portfolio,
    cm.status,
    cm.status_novo,
    cast(cm.id_atendimento as string) as id_evento,
    cm.id_partner_transfer,
    cm.id_procedimento,
    cm.numero_atendimento as id_atendimento,
    cm.ds_grupo_procedimento,
    cm.qtd_solicitada_prestador,
    cm.data_origem as data_hora_solicitacao,
    cm.data_origem_date as data_solicitacao,
    cm.prestador_executor_id,
    cm.prestador_executor_nome as prestador_executor_nome,
    rc1.cnpj as CNPJ_prestador_executor,
    rc1.cpf as CPF_prestador_executor,
    rc1.dt_credenciamento as dt_credenciamento_prestador_executor,
    rc1.dt_descredenciamento as dt_descredenciamento_prestador_executor,
    rc1.status_prestador as status_atual_prestador_executor,
    coalesce(bcc.classificacao, 'N/I') as classificacao_prestador_executor,
    rc1.cep as cep_prestador_executor,
    cm.prestador_solicitante_id,
    cm.prestador_solicitante_nome as prestador_solicitante_nome,
    rc2.cnpj as CNPJ_prestador_solicitante,
    rc2.cpf as CPF_prestador_solicitante,
    rc2.dt_credenciamento as dt_credenciamento_prestador_solicitante,
    rc2.dt_descredenciamento as dt_descredenciamento_prestador_solicitante,
    rc2.status_prestador as status_atual_prestador_solicitante,
    cm.local_pgmnto_copart,
    cm.valor_copart,
    cm.pet_idade_proced,
    cm.fl_compra_adicional,
    cm.valor_origem,
    cm.valor_autorizado,
    cm.valor_autorizado_novo,
    cm.valor_glosado_novo,
    cm.fl_glosa_definitiva,
    cm.status_glosa,
    cast(cm.motivo_glosa_id as string) as motivo_glosa_id,
    cm.motivo_glosa_desc as descritivo_glosa,
    cm.vl_bruto
  from op_health.custos_medicos cm
  -- Dados do prestador executor
  left join op_health.dts_rede_credenciada rc1
    on cm.prestador_executor_id = rc1.id
  -- Dados do prestador solicitante
  left join op_health.dts_rede_credenciada rc2
    on cm.prestador_solicitante_id = rc2.id
  left join op_health.beneficiarios ben
    on cm.id_pet_bu = ben.id_pet_bu
  -- Nessa CTE pegaremos informações do plano (Importante não trazer da beneficiários pois lá só temos o plano atual e na custos_medicos consideramos o plano histórico do pet)
  left join curated_health.health_plan pl
    on cm.id_plano = pl.id
    and pl.refdate <= current_date()
  -- Informação da região do plano (Olhando para plano da custos_medicos)
  left join curated_health.health_regions re
    on pl.region_id = re.id
    and re.refdate <= current_date()
  -- Informação do preço mensal do plano (Olhando para plano da custos_medicos)
  left join curated_health.health_plan_price pp
    on pl.id = pp.plan_id
    and pp.refdate <= current_date()
  left join cnpj_cpf_empresa_titular ccet
    on ben.id_pet = ccet.id_pet
  left join base_classificacao_clinicas bcc
    on cm.prestador_executor_id = bcc.id_prestador
where sistema_origem_custo = 'ERP Nofaro' -- Somente ERP da Nofaro
  and status = 'Ativo' -- Somente Ativos
  and data_origem_date between '2024-12-01' and '2024-12-31'
),
final_1 as (
  -- Construindo algumas flag's e join com o funil de auditoria analítico
  select
    bf.*,
    -- No campo abaixo quis diferenciar o que temos histórico do que não temos
    -- Os nulos são os casos sem histórico, os 0 e 1 são o que temos histórico
    -- Estar nulo e estar com 0 devem ser tratados de forma diferente, por isso fiz essa diferenciação
    case when data_solicitacao < '2024-01-01' then null
         when data_solicitacao > '2024-01-01' and faa.fl_em_analise is null then 0
         else faa.fl_em_analise
         end as fl_caiu_regra_sistema,
    -- Abaixo criaremos a flag de glosa
    -- Para tratar o histórico, vou considerar que glosamos tudo que está com o status de glosado irrecursável, o que está recursado e os status referentes a análise de recurso (Só analisamos um recurso pois o procedimento foi glosado)
    -- case
    --   when bf.data_solicitacao < '2024-01-01'
    --     and bf.status_novo in ('Glosado Irrecursável',
    --                            'Glosado Recursável',
    --                            'Recursado', 'Recurso aceito',
    --                            'Recurso recusado',
    --                            'Fora prazo recurso',
    --                            'Recurso aceito parcialmente')
    --     then 1
    --   when bf.data_solicitacao < '2024-01-01'
    --     and bf.status_novo not in ('Glosado Irrecursável',
    --                                'Glosado Recursável',
    --                                'Recursado',
    --                                'Recurso aceito',
    --                                'Recurso recusado',
    --                                'Fora prazo recurso',
    --                                'Recurso aceito parcialmente')
    --     then 0
    --   when bf.data_solicitacao >= '2024-01-01'
    --     and faa.fl_glosado is null
    --     and status_glosa = 'Glosa Definitiva'
    --     then 1 -- Tratar casos que entram em recurso sem passarem pelo status de glosa (olhar dados nessa planilha: https://docs.google.com/spreadsheets/d/1sZMcTIRmCIaXYQ3qjeXVDpPQ1vwA3CFG-Ag6RVnqQJI/edit#gid=880209354)   
    --   when bf.data_solicitacao >= '2024-01-01' and faa.fl_glosado is null then 0
    --   when bf.data_solicitacao >= '2024-01-01' and faa.fl_glosado is not null then faa.fl_glosado
    --   end as fl_glosa, -- Não utilizo mais (Tombei para o campo ds_motivo_glosa em 02/04/2025 para equalizar acompanhamentos => Ainda está pendente alteração no funil de auditoria, mas já está nessa base histórica e no dash da Neurotech) 
      -- faa.fl_glosado,
      -- Agora vamos construir a flag de recurso
      -- Pegamos o que foi recursado a partir da tabela partner_transfer_request (Onde cai todas os recursos)
      -- Essa tabela começou a ser abastecida em 12/2022, então só temos o histórico a partir dessa data
      case when bf.data_solicitacao < '2024-01-01' and ar.id_partner_transfer is not null then 1
           when bf.data_solicitacao < '2024-01-01' and ar.id_partner_transfer is null then 0
           when bf.data_solicitacao >= '2024-01-01' and faa.fl_recursado is null then 0
           when bf.data_solicitacao >= '2024-01-01' and faa.fl_recursado is not null then faa.fl_recursado
      end as fl_recurso, -- Não utilizo mais (Tombei para o funil em 02/04/2025 para eualizar acompanhamentos)
      faa.fl_recursado, -- Adicionado em 02/04/2025 para equalizar conceitos com outros acompanhamentos
      faa.fl_auditoria -- Adicionado em 02/04/2025 para equalizar conceitos com outros acompanhamentos + pedido da Neurotech
  from final_0 bf
  -- Trazer informações do funil
  left join op_health.funil_auditoria_analitico faa
    on cast(bf.id_partner_transfer as int64) = faa.id_partner_transfer
  left join atendimentos_recursados ar
    on bf.id_partner_transfer = ar.id_partner_transfer
  where especie is not null -- Excluindo 62 casos de teste
  -- Colocar flag do que caiu em análise
  -- Casos que foram glosados sem cair em análise
),
final_2 as (
  select
  id_evento,
  id_procedimento,
  id_atendimento,
  ds_grupo_procedimento,
  qtd_solicitada_prestador,
  data_hora_solicitacao,
  data_solicitacao,
  data_solicitacao as data_origem, --mesma data de solicitacao
  data_solicitacao as data_aviso_prestador, --mesma data de solicitacao
  case when CNPJ_prestador_executor is not null and CPF_prestador_executor is not null
       then CNPJ_prestador_executor
       else CNPJ_prestador_executor
       end as CNPJ_prestador_executor,
  case when CNPJ_prestador_executor is not null and CPF_prestador_executor is not null
       then null
       else CPF_prestador_executor
      end as CPF_prestador_executor,
  cep_prestador_executor,
  dt_credenciamento_prestador_executor,
  dt_descredenciamento_prestador_executor,
  status_atual_prestador_executor,
  classificacao_prestador_executor,
  cast(prestador_executor_id as string) as prestador_executor_id,
  prestador_executor_nome,
  case when CNPJ_prestador_solicitante is not null and CPF_prestador_solicitante is not null
       then CNPJ_prestador_solicitante
       else CNPJ_prestador_solicitante
       end as CNPJ_prestador_solicitante,
  case when CNPJ_prestador_solicitante is not null and CPF_prestador_solicitante is not null
       then null
       else CPF_prestador_solicitante
       end as CPF_prestador_solicitante,
  -- CPF_prestador_solicitante,
  dt_credenciamento_prestador_solicitante,
  dt_descredenciamento_prestador_solicitante,
  status_atual_prestador_solicitante,
  cast(prestador_solicitante_id as string) as prestador_solicitante_id,
  prestador_solicitante_nome,
  coalesce(round(valor_copart, 2), 0) as valor_copart,
  local_pgmnto_copart,
  pet_idade_proced,
  fl_compra_adicional,
  coalesce(round(valor_origem, 2), 0) as valor_origem,
  coalesce(round(valor_autorizado, 2), 0) as valor_autorizado,
  coalesce(round(valor_autorizado_novo, 2), 0) as valor_autorizado_novo,
  -- Temos casos com problema processual, ou seja, o recurso acontece mesmo sem uma glosa registrada (Tanto nos casos com a marcação do funil quanto nos mais antigos)
  fl_auditoria,
  if(descritivo_glosa is not null, 1, 0) as fl_glosa,
  coalesce(motivo_glosa_id, 'N/I') as motivo_glosa_id,
  coalesce(descritivo_glosa, 'N/I') as motivo_glosa,
  fl_recursado as fl_recurso,
  --data_glosa, não temos?
  --motivo_glosa, mesmo campo do descritivo_glosa
  fl_glosa_definitiva,
  status_glosa,
  fl_caiu_regra_sistema,
  coalesce(valor_glosado_novo, 0) as valor_glosado_novo,
  --tipo_regra,
  empresa,
  sistema_origem_custo,
  portfolio,
  empresa_titular,
  CNPJ_empresa_titular,
  cpf_tutor,
  cpf_tutor_hash,
  cidade_tutor,
  estado_tutor,
  tutor_cep,
  nome_plano,
  nome_plano_dts_pet,
  regiao_plano,
  uf_plano,
  id_plano,
  valor_mensal_plano,
  --cd_cotnrato, não temos!
  id_pet_bu,
  dt_nascimento_pet,
  idade_pet_anos,
  especie,
  nome_pet,
  raca_pet,
  genero_pet,
  dt_compra,
  status,
  status_novo,
  vl_bruto
  --id_partner_transfer, retirar da base
  from final_1
),
final as (
select distinct
       *,
       case when id_procedimento in ('115','608','6','373','116','368','155','146','375','33','5','29','14','28','631','3','217','351','374','140','204','612','499','632','93','639') then 1
            when vl_bruto <= 0 then 1
            when fl_compra_adicional = '1' then 1
            else 0
       end as fl_desconsiderar_score
from final_2
order by id_pet_bu, data_hora_solicitacao desc
)
select *
from final
"""

query_bytes(query, client)

c:\Users\anderson.paiva\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Quer matar o Cesar????


'8.49 GB'

In [5]:
## 2. Funções Auxiliares

### 2.1 Função de Execução de Query

def run_query(query, client):
    """
    Executa uma query no BigQuery e retorna um DataFrame com tipos de dados apropriados.
    
    Args:
        query (str): Query SQL a ser executada
        client: Cliente do BigQuery
        
    Returns:
        pd.DataFrame: DataFrame com os resultados da query
    """
    try:
        print("Iniciando a consulta:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        query_job = client.query(query)
        results = query_job.result()
        
        # Convertendo para DataFrame com tipos de dados apropriados
        df = pd.DataFrame(
            [dict(row) for row in results],
            dtype={
                'id_evento': str,
                'id_procedimento': str,
                'id_atendimento': str,
                'qtd_solicitada_prestador': float,
                'valor_copart': float,
                'valor_origem': float,
                'valor_autorizado': float,
                'valor_autorizado_novo': float,
                'valor_glosado_novo': float,
                'vl_bruto': float,
                'fl_glosa': int,
                'fl_recurso': int,
                'fl_glosa_definitiva': int,
                'fl_caiu_regra_sistema': int,
                'fl_desconsiderar_score': int,
                'idade_pet_anos': float,
                'valor_mensal_plano': float
            }
        )
        
        print("Consulta finalizada com sucesso!")
        return df
        
    except Exception as e:
        print(f"Erro ao executar a consulta: {str(e)}")
        raise

In [6]:
df = run_query(query, client)

Iniciando a consulta: 2025-04-03 19:12:38
Erro ao executar a consulta: object of type 'type' has no len()


TypeError: object of type 'type' has no len()

In [ ]:
df.info()

In [ ]:
print(df)

In [ ]:
save_to_csv(df, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'hist_neurotech_202412')

------------------------